In [4]:
import pandas as pd
import numpy as np
import matplotlib as plt

seds = pd.read_csv('../datasets/seds.csv') #1
revenue_to_provider = pd.read_csv('../datasets/revenue_to_provider.csv') #2
filter_pack_concentrations_weekly = pd.read_csv('../datasets/filter_pack_concentration_weekly.csv') #3
energy_workforce_training = pd.read_csv('../datasets/energy_workforce_training.csv') #4
vehicles = pd.read_csv('../datasets/vehicles.csv') #5
land_temp_by_city = pd.read_csv('../datasets/land_temp_by_city.csv') #6
land_temp_by_state = pd.read_csv('../datasets/land_temp_by_state.csv') #7
power_plants = pd.read_csv('../datasets/power_plants.csv') #8

In [5]:
seds['energy_source'] = seds['msn'].apply(lambda v: v[:2])
seds['energy_sector'] = seds['msn'].apply(lambda v: v[2:4])
seds['energy_unit'] = seds['msn'].apply(lambda v: v[4])

In [7]:
print(len(set(power_plants.city)))
print(len(set(land_temp_by_city.city)))

3742
248


In [8]:
# add state_name column to dataset_6

power_plant_cities = dict()
for index, row in power_plants.iterrows():
    if (row['city']) not in power_plant_cities.keys():
        power_plant_cities[row['city']] = [(row['state_name'],row['latitude'], row['longitude'])]
    else:
        already_in = False
        for el in power_plant_cities[row['city']]:
            if el[0] == row['state_name']:
                already_in = True
        if not already_in:
            power_plant_cities[row['city']].append((row['state_name'],row['latitude'], row['longitude']))
            

def get_state(row, cities_dict):
    try:
        if len(cities_dict[row['city']])==1:
            return cities_dict[row['city']][0][0]
        else:
            return min(cities_dict[row['city']], lambda v: (v[0]-row['latitude'])**2 + (v[1]-row['longitude']**2))[0]
    except:
        return 'UNDEFINED'

land_temp_by_city['state_name'] = land_temp_by_city.apply(get_state,args=(power_plant_cities,), axis=1)


In [10]:
# create dictionaries with cities from dataset 6 and 8

from math import cos, asin, sqrt

temp_cities_dict = dict()
for index, row in land_temp_by_city.iterrows():
    if (row['city'], row['state_name']) not in temp_cities_dict.keys():
        temp_cities_dict[(row['city'], row['state_name'])] = (row['latitude'], row['longitude'])
        
temp_cities = []
for key, item in temp_cities_dict.items():
    temp_cities.append({'city_state':key, 'latitude':item[0], 'longitude':item[1]})
    
power_cities_dict = dict()
for index, row in power_plants.iterrows():
    if (row['city'], row['state_name']) not in power_cities_dict.keys():
        power_cities_dict[(row['city'], row['state_name'])] = (row['latitude'], row['longitude'])
        
power_cities = []
for key, item in power_cities_dict.items():
    power_cities.append({'city_state':key, 'latitude':item[0], 'longitude':item[1]})
        
        

In [11]:
def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(v, data): # data = cities, v = row 
    x = min(data, key=lambda p: distance(v['latitude'],v['longitude'],p['latitude'],p['longitude']))
    return pd.Series([x['city_state'], distance(x['latitude'],x['longitude'],v['latitude'], v['longitude'])])

In [ ]:
filter_pack_concentrations_weekly[['temp_city', 'temp_city_dist']] = filter_pack_concentrations_weekly.apply(closest, args=(temp_cities,), axis=1)
filter_pack_concentrations_weekly[['power_city', 'power_city_dist']] = filter_pack_concentrations_weekly.apply(closest, args=(power_cities,), axis=1)

